In [0]:
from pyspark.sql.functions import *
import re


#### Landing to Bronze

In [0]:
#Read streaming table with cloudFiles Format
df_raw_stream = (
    spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format", "csv")
    .option("cloudFiles.inferColumnTypes", "true") #Infer column type for new columns 
    .option("cloudFiles.schemaLocation","/Volumes/workspace/default/schema_tracking")
    .option("header", "true")
    .option("cloudFiles.schemaEvolutionMode", "addNewColumns") #Add new columns to schema
    .load("/Volumes/workspace/default/data_stage")
)


#Normalize columns
def normalize_cols(cols):
    import re, unicodedata
    out = []
    for c in cols:
        s = unicodedata.normalize('NFKD', c).encode('ascii', 'ignore').decode('ascii')  
        s = re.sub(r"\s*\(.*?\)\s*", "", s)                                          
        s = re.sub(r"[^\w]+", "_", s).strip("_")                                      
        out.append(s.upper())
    return out

bronze_renamed = df_raw_stream.toDF(*normalize_cols(df_raw_stream.columns))
# write stream
(brownze_q := bronze_renamed.writeStream
  .format("delta")
  .option("checkpointLocation","/Volumes/workspace/pmx_etl/checkpoints_bronze")
  .option("mergeSchema", "true")
  .trigger(once=True) 
  .table("pmx_etl.bronze_pmx_prod")
)

In [0]:
#dbutils.fs.rm("/Volumes/workspace/pmx_etl/checkpoints_bronze", recurse=True)
#spark.sql("""
#          TRUNCATE TABLE pmx_etl.bronze_pmx_prod
#          """)